In [2]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [3]:
#hide warnings
HTML('''<script>
code_show_err=true; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output errors, click <a href="javascript:code_toggle_err()">here</a>.''')

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
print('A99_Data2: Comparing responses in PAM neurons when driven by SS12984 using different stim periods and intensities.')


A99_Data2: Comparing responses in PAM neurons when driven by SS12984 using different stim periods and intensities.


In [110]:
import sys
sys.path.append('/home/daniel/Desktop/ResearchUbuntuYoga720/A30_FastROI/CircuitCatcher2')
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import ccModules as cc
import ccModules2 as cc2
import pandas as pd
from collections import defaultdict
from IPython.display import IFrame
from bokeh.palettes import Spectral4
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import Range1d, Span, Legend, BoxAnnotation
from bokeh.palettes import Category20 as palette
import itertools  
output_notebook()

Loading BokehJS ...

In [112]:
#get the pickle data containing intensity data
targetfolder = '/media/daniel/Seagate Backup Plus Drive/A99/A99_Data2'
outputfolder = 'Analysis20190617'
output_pickle_file = 'A99_Data2_CompiledIntensityData.pkl'
path_to_pickle = str(Path(targetfolder)/ outputfolder / output_pickle_file)
data = pd.read_pickle(path_to_pickle)

In [113]:

def pullIntensityData2(dataframe, rois):
    #dataframe = pandas dataframe with intensity data in column as a dictionary
    #rois = [desired rois]
    list_of_rois = []
    all_rois = []
    for row, dseries in dataframe.iterrows():
        intensity_data=pd.DataFrame(dseries['intensity_data'])
        background = intensity_data[intensity_data['Name'].str.contains('Background')]
        background_intensity = np.mean(np.vstack(background['intensity']), axis =0)
        dseries.drop(['intensity_data'], inplace=True)
        series_dict = dseries.to_dict()
        for croi in rois:
            #print(croi)
            cframe = intensity_data[intensity_data['Name'].str.contains(croi)]
            intensity_background = np.mean(np.vstack(cframe['intensity']), axis = 0)
            series_dict['intensity-background'] = intensity_background
            series_dict['roi'] = croi
            list_of_rois.append(series_dict.copy())
    data2 = pd.DataFrame(list_of_rois)
    #return data2
    return data2



In [114]:
#pulling out individual rois and converting them into rows
rois =['a1', "b'p2a", "b'p2mp", 'b1', 'b2', 'g3', 'g4', 'g5']
#pull out roi data and add each roi as separate row
data_frame = pullIntensityData2(data, rois)

In [115]:
#add colors to roi
colors = itertools.cycle(palette[20])
roi_colors = dict(zip(rois, colors))

In [119]:
print('Making sure all the rois have the correct time length for each assay. Some assays are short')
expected_duration = {1:40, 2:50, 4:70, 8:110, 16:190, 32:350}
#check if the timestamp matches the expected value
matches = []
for row, dseries in data_frame.iterrows():
    matches.append(dseries['timestamp'].shape[0] == expected_duration[dseries['Stim_Period']])
print('There were ' + str(matches.count(False)) + ' where the time does not match the expected time period of ' + str(data_frame.shape[0]) + ' instances.')
data_frame = data_frame[matches]

Making sure all the rois have the correct time length for each assay. Some assays are short
There were 104 where the time does not match the expected time period of 1032 instances.


In [120]:
#get the counts
print("The numbers tested for a single roi a1L,")
comparisons = ['LED_Intensity', 'Stim_Period', 'roi']
groups = data_frame[data_frame['roi'] == 'a1'].groupby(comparisons)
groups['roi'].count()

The numbers tested for a single roi a1L,


LED_Intensity  Stim_Period  roi
0.2            1            a1     7
               2            a1     7
               4            a1     7
               8            a1     7
               16           a1     7
               32           a1     7
0.6            1            a1     5
               2            a1     5
               4            a1     5
               8            a1     5
               16           a1     6
               32           a1     6
1.8            1            a1     7
               2            a1     7
               4            a1     7
               8            a1     7
               16           a1     7
               32           a1     7
Name: roi, dtype: int64

In [121]:
def voltage_intervals(matched_voltage):
    diff = np.diff(matched_voltage)
    down_index = np.where(diff <0)[0]
    up_index = np.where(diff >0)[0]
    stim_start_stops =list(zip(up_index, down_index))
    return stim_start_stops

In [122]:
print('Graphs comparing normalized mean intensity across rois keeping stim intensity and duration constant')
for cgroup, cframe in data_frame.groupby(['LED_Intensity', 'Stim_Period', ]):
    #get the min and max intensity
    stats_dict = defaultdict(dict)
    timestamp = np.mean(np.hstack(cframe['timestamp'].values), axis=1)
    voltage = np.mean(np.hstack(cframe['voltage'].values), axis=1)
    for croi, cframe2 in cframe.groupby(['roi']):
        normalized_intensity = np.vstack(cframe2['intensity-background'].values) / np.mean(np.vstack(cframe2['intensity-background'].values))
        stats_dict[croi]['mean'] = np.mean(normalized_intensity, axis=0)
        stats_dict[croi]['se'] = np.std(normalized_intensity, axis=0) / np.sqrt(cframe2['intensity-background'].shape[0])
    stats_frame = pd.DataFrame(stats_dict)
    p = figure(plot_width=800, plot_height=400, y_range=Range1d(np.vstack(stats_frame.loc['mean']).min(), np.vstack(stats_frame.loc['mean']).max()), x_range = Range1d(0, np.max(timestamp)), x_axis_label = 'Time (S)', y_axis_label = 'Intensity - Background')
    p.title.text = 'LED intensity (V) : ' + str(cgroup[0]) + ' ; ' + 'Stim Period Duration (s) : ' + str(cgroup[1])
    legend_it = []
    stim_start_stops = voltage_intervals(voltage)
    for pt in stim_start_stops: 
        stim_time = BoxAnnotation(left = pt[0]/100, right = pt[1]/100,  line_alpha=0.1 , fill_color = (255, 215, 0))
        p.add_layout(stim_time)
    
    #draw individual rois
    for croi, dseries in stats_frame.T.iterrows():
        c = p.line(x = timestamp, y = dseries['mean'], line_color = roi_colors[croi], line_width =2)
        legend_it.append((croi, [c]))
    legend = Legend(items = legend_it, location=(0, 100))
    legend.click_policy="hide"
    p.add_layout(legend, 'right')

    show(p)

Graphs comparing normalized mean intensity across rois keeping stim intensity and duration constant


In [125]:
#add colors to roi
LED_intensities = data_frame['LED_Intensity'].unique()
colors = itertools.cycle(palette[20])
LED_colors = dict(zip(LED_intensities, colors))

In [130]:
print('Graphs comparing normalized mean intensity across stim intensities keeping roi and duration constant')
for cgroup, cframe in data_frame.groupby(['roi', 'Stim_Period', ]):
    #get the min and max intensity
    stats_dict = defaultdict(dict)
    timestamp = np.mean(np.hstack(cframe['timestamp'].values), axis=1)
    voltage = np.mean(np.hstack(cframe['voltage'].values), axis=1)
    for croi, cframe2 in cframe.groupby(['LED_Intensity']):
        normalized_intensity = np.vstack(cframe2['intensity-background'].values) / np.mean(np.vstack(cframe2['intensity-background'].values))
        stats_dict[croi]['mean'] = np.mean(normalized_intensity, axis=0)
        stats_dict[croi]['se'] = np.std(normalized_intensity, axis=0) / np.sqrt(cframe2['intensity-background'].shape[0])
    stats_frame = pd.DataFrame(stats_dict)
    p = figure(plot_width=800, plot_height=400, y_range=Range1d(np.vstack(stats_frame.loc['mean']).min(), np.vstack(stats_frame.loc['mean']).max()), x_range = Range1d(0, np.max(timestamp)), x_axis_label = 'Time (S)', y_axis_label = 'Intensity - Background')
    p.title.text = 'ROI : ' + str(cgroup[0]) + ' ; ' + 'Stim Period Duration (s) : ' + str(cgroup[1])
    legend_it = []
    stim_start_stops = voltage_intervals(voltage)
    for pt in stim_start_stops: 
        stim_time = BoxAnnotation(left = pt[0]/100, right = pt[1]/100,  line_alpha=0.1 , fill_color = (255, 215, 0))
        p.add_layout(stim_time)
    
    #draw individual rois
    for croi, dseries in stats_frame.T.iterrows():
        c = p.line(x = timestamp, y = dseries['mean'], line_color = LED_colors[croi], line_width =2)
        legend_it.append((str(croi), [c]))
    legend = Legend(items = legend_it, location=(0, 100))
    legend.click_policy="hide"
    p.add_layout(legend, 'right')

    show(p)

Graphs comparing normalized mean intensity across stim intensities keeping roi and duration constant


In [132]:
!jupyter-nbconvert D2_A99_Data2_CreateGraphs.ipynb

[NbConvertApp] Converting notebook D2_A99_Data2_CreateGraphs.ipynb to html
[NbConvertApp] Writing 1804249 bytes to D2_A99_Data2_CreateGraphs.html
